# Exploring Airbnb Market Trends

![NYC Skyline](nyc.jpg)

Welcome to New York City, one of the most-visited cities in the world. There are many Airbnb listings in New York City to meet the high demand for temporary lodging for travelers, which can be anywhere between a few nights to many months. In this project, I (Saddam) will take a closer look at the New York Airbnb market by combining data from multiple file types like `.csv`, `.tsv`, and `.xlsx`.

Recall that **CSV**, **TSV**, and **Excel** files are three common formats for storing data. 
Three files containing data on 2019 Airbnb listings are available to you:

**data/airbnb_price.csv**
This is a CSV file containing data on Airbnb listing prices and locations.
- **`listing_id`**: unique identifier of listing
- **`price`**: nightly listing price in USD
- **`nbhood_full`**: name of borough and neighborhood where listing is located

**data/airbnb_room_type.xlsx**
This is an Excel file containing data on Airbnb listing descriptions and room types.
- **`listing_id`**: unique identifier of listing
- **`description`**: listing description
- **`room_type`**: Airbnb has three types of rooms: shared rooms, private rooms, and entire homes/apartments

**data/airbnb_last_review.tsv**
This is a TSV file containing data on Airbnb host names and review dates.
- **`listing_id`**: unique identifier of listing
- **`host_name`**: name of listing host
- **`last_review`**: date when the listing was last reviewed

In [106]:
# Importing necessary packages
import pandas as pd
import numpy as np

from thefuzz import process

As a consultant working for a real estate start-up, you have collected Airbnb listing data from various sources to investigate the short-term rental market in New York. You'll analyze this data to provide insights on private rooms to the real estate company.

## There are three files in the data folder: `airbnb_price.csv`, `airbnb_room_type.xlsx`, `airbnb_last_review.tsv`.

In [107]:
airbnb_price = pd.read_csv('data/airbnb_price.csv') 
airbnb_room_type = pd.ExcelFile('data/airbnb_room_type.xlsx').parse('airbnb_room_type')
airbnb_last_review = pd.read_csv('data/airbnb_last_review.tsv',sep='\t')

In [108]:
airbnb_price.head()

,listing_id,price,nbhood_full
0,2595,225 dollars,"Manhattan, Midtown"
1,3831,89 dollars,"Brooklyn, Clinton Hill"
2,5099,200 dollars,"Manhattan, Murray Hill"
3,5178,79 dollars,"Manhattan, Hell's Kitchen"
4,5238,150 dollars,"Manhattan, Chinatown"


In [109]:
airbnb_room_type.head()

,listing_id,description,room_type
0,2595,Skylit Midtown Castle,Entire home/apt
1,3831,Cozy Entire Floor of Brownstone,Entire home/apt
2,5099,Large Cozy 1 BR Apartment In Midtown East,Entire home/apt
3,5178,Large Furnished Room Near B'way,private room
4,5238,Cute & Cozy Lower East Side 1 bdrm,Entire home/apt


In [110]:
airbnb_last_review.head()

,listing_id,host_name,last_review
0,2595,Jennifer,May 21 2019
1,3831,LisaRoxanne,July 05 2019
2,5099,Chris,June 22 2019
3,5178,Shunichi,June 24 2019
4,5238,Ben,June 09 2019


## 1. Joining data from all sources into one DataFrame

In [111]:
# join data on listing_id in this order (( df1 merge df2 ) merge df3 )
join = airbnb_price.merge(airbnb_room_type,on='listing_id',how='inner')
df = join.merge(airbnb_last_review,on='listing_id',how='inner')

df.head()

,listing_id,price,nbhood_full,description,room_type,host_name,last_review
0,2595,225 dollars,"Manhattan, Midtown",Skylit Midtown Castle,Entire home/apt,Jennifer,May 21 2019
1,3831,89 dollars,"Brooklyn, Clinton Hill",Cozy Entire Floor of Brownstone,Entire home/apt,LisaRoxanne,July 05 2019
2,5099,200 dollars,"Manhattan, Murray Hill",Large Cozy 1 BR Apartment In Midtown East,Entire home/apt,Chris,June 22 2019
3,5178,79 dollars,"Manhattan, Hell's Kitchen",Large Furnished Room Near B'way,private room,Shunichi,June 24 2019
4,5238,150 dollars,"Manhattan, Chinatown",Cute & Cozy Lower East Side 1 bdrm,Entire home/apt,Ben,June 09 2019


## 2. What are the dates of the earliest and most recent reviews? Store these values as two separate variables with your preferred names.

In [112]:
# convert and format review column
df['last_review'] =  pd.to_datetime(df['last_review'], format='%B %d %Y').dt.date

In [113]:
# get earliest and most recent reviews
earliest = df['last_review'].min()
latest = df['last_review'].max()

print(f'The earliest review was on {earliest}')
print(f'The latest review was on {latest}')

The earliest review was on 2019-01-01
The latest review was on 2019-07-09


## 3. How many of the listings are private rooms? Save this into any variable.

In [114]:
# room categories before eliminating inconsistent categories
df['room_type'].unique()

array(['Entire home/apt', 'private room', 'Private room',
       'entire home/apt', 'PRIVATE ROOM', 'shared room',
       'ENTIRE HOME/APT', 'Shared room', 'SHARED ROOM'], dtype=object)

In [115]:
# actuall categories of rooms
room_categories = ['private room', 'entire home/apt', 'shared room']

# matching and update the room_type using 'thefuzz' lib
for index in range(0,df.shape[0]):
    df.loc[index,'room_type'] = process.extract(\
                                                df.loc[index,'room_type'],\
                                                room_categories,limit=1\
                                               )[0][0]

In [116]:
# room categories after using method 'Levenshtein distance' b/w strings
df['room_type'].unique()

array(['entire home/apt', 'private room', 'shared room'], dtype=object)

In [117]:
# now actual number of private rooms
no_private_rooms = df.query("room_type=='private room'").shape[0]

print(f'There are {no_private_rooms} private rooms in the listings.')

There are 11356 private rooms in the listings.


## 4. What is the average listing price? Round to the nearest two decimal places and save into a variable.

In [118]:
# removing everything which is not a digit and convert to int
df['price'] = df['price'].str.replace(r'\D+','').astype(int)

In [119]:
# calculate mean and round to 2 decimal places
avg_listing_price = np.round(df['price'].mean(),2)

print(f'Average listing price is {avg_listing_price}')

Average listing price is 141.78


## 5. Combine the new variables into one DataFrame called review_dates with four columns in the following order: `first_reviewed`, `last_reviewed`, `nb_private_rooms`, and `avg_price`. The DataFrame should only contain one row of values.

In [120]:
data = {'first_reviewed': [earliest], 'last_reviewed': [latest],\
        'nb_private_rooms':[no_private_rooms],'avg_price':[avg_listing_price]}

review_dates = pd.DataFrame(data)

print(review_dates)

  first_reviewed last_reviewed  nb_private_rooms  avg_price
0     2019-01-01    2019-07-09             11356     141.78
